# FAIIA-IDS Ablation Study (Refactored)

This notebook runs the ablation study for the FAIIA-IDS model by cloning the refactored codebase from GitHub.

**Note:** Please replace `https://github.com/USERNAME/REPO_NAME.git` with your actual repository URL.

In [ ]:
# 1. Clone Repository
# TODO: Replace with your actual repository URL
GIT_REPO_URL = "https://github.com/Arif-Foysal/FAA-Net.git"
REPO_DIR = "FAA-Net" # This usually matches the name of the git repo

!git clone {GIT_REPO_URL}

import os
if os.path.exists(REPO_DIR):
    os.chdir(REPO_DIR)
    print(f"Changed directory to: {os.getcwd()}")
else:
    print(f"Warning: Could not find directory {REPO_DIR}. Check if git clone succeeded.")

# 2. Mount Google Drive (for saving models)
try:
    from google.colab import drive
    drive.mount('/content/drive')
    print("Google Drive mounted successfully.")
except ImportError:
    print("Not running in Google Colab, skipping Drive mount.")

# 3. Install Dependencies
!pip install -r requirements.txt

In [ ]:
import sys
# Ensure project root is in path
if os.getcwd() not in sys.path:
    sys.path.append(os.getcwd())

import torch
import pandas as pd
from scripts.run_ablation import main as run_ablation_experiment
from scripts.train_main import main as train_main_model
from scripts.train_baselines import main as run_baseline_experiment

## 1. Train Main EDAN v3 Model

Trains the full EDAN v3 model with FAIIA and Minority Prototypes using the standard configuration.

In [ ]:
try:
    # Run training logic
    train_main_model()
except FileNotFoundError as e:
    print(f"Error: {e}")
    print("Please ensure dataset files (UNSW_NB15) are present in the project root or /content.")

## 2. Run Ablation Study

Runs 6 experiments:
1. Vanilla DNN + BCE
2. Vanilla DNN + Focal Loss
3. FAIIA + BCE
4. FAIIA + Focal Loss
5. **FAIIA + EWKM + BCE** *(NEW — Entropy-Weighted KMeans prototypes)*
6. **FAIIA + EWKM + Focal Loss** *(NEW — Entropy-Weighted KMeans prototypes)*

In [ ]:
try:
    run_ablation_experiment()
except FileNotFoundError as e:
    print(f"Error: {e}")

## 3. Run Standard Baselines

Runs classical ML baselines:
1. XGBoost
2. LightGBM

In [ ]:
try:
    run_baseline_experiment()
except FileNotFoundError as e:
    print(f"Error: {e}")

## 4. View Consolidated Results

Load and display the summary CSVs generated by the experiments.

In [ ]:
results = []

if os.path.exists('ablation_summary.csv'):
    df_ablation = pd.read_csv('ablation_summary.csv', index_col=0)
    print("Ablation Results Loaded")
    results.append(df_ablation)

if os.path.exists('baseline_summary.csv'):
    df_baseline = pd.read_csv('baseline_summary.csv', index_col=0)
    print("Baseline Results Loaded")
    results.append(df_baseline)
    
if results:
    final_df = pd.concat(results)
    display(final_df)
else:
    print("No results files found. Ensure experiments ran successfully.")